In [1]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf


In [2]:
tf.__version__

'2.1.0'

In [3]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Lambda, Conv2D
from tensorflow.keras.models import load_model, Model


In [4]:
def yolo_filter_boxes(
    box_confidence, boxes, box_class_probs, threshold=0.6
):
    box_scores = tf.multiply(box_confidence, box_class_probs)
    box_classes = tf.math.argmax(box_scores, axis=-1)
    box_class_scores = np.max(box_scores, axis=-1)
    
    filtering_mask = (box_class_scores >= threshold)
    
    print(box_scores.shape, box_classes.shape, box_class_scores.shape, filtering_mask.shape, boxes.shape)
    
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)
    
    
    return scores, boxes, classes

In [5]:
box_confidence = tf.random.normal([19, 19, 5, 1], mean=1, stddev=4, seed=1)
boxes = tf.random.normal([19, 19, 5, 4], mean=1, stddev=4, seed = 1)
box_class_probs = tf.random.normal([19, 19, 5, 80], mean=1, stddev=4, seed=1)

In [6]:
scores, boxes, classes = yolo_filter_boxes(
    box_confidence, boxes, box_class_probs, threshold=0.5
)

(19, 19, 5, 80) (19, 19, 5) (19, 19, 5) (19, 19, 5) (19, 19, 5, 4)


In [7]:
print("scores[2] = " + str(tf.keras.backend.eval(scores[2])))
print("boxes[2] = " + str(tf.keras.backend.eval(boxes[2])))
print("classes[2] = " + str(tf.keras.backend.eval(classes[2])))
print("scores.shape = " + str(scores.shape))
print("boxes.shape = " + str(boxes.shape))
print("classes.shape = " + str(classes.shape))

scores[2] = 16.064978
boxes[2] = [ 1.0273384 -2.1178942  4.8872733 -4.0143332]
classes[2] = 78
scores.shape = (1786,)
boxes.shape = (1786, 4)
classes.shape = (1786,)


## Intersection over Union

In [8]:
def IOU(box1, box2):
    
    xi1 = max([box1[0], box2[0]])
    yi1 = max([box1[1], box2[1]])
    xi2 = min([box1[2], box2[2]])
    yi2 = min([box1[3], box2[3]])
    
    inter_area = abs(yi1 - yi2) * abs(xi1 - xi2)
    
    box1_area = abs(box1[3] - box1[1]) * abs(box1[2] - box1[0])
    box2_area = abs(box2[3] - box2[1]) * abs(box2[2] - box2[0])
    
    inter_area = abs(xi1 - xi2) * abs(yi1 - yi2)
    
    union_area = box1_area + box2_area - inter_area
    
    return inter_area / union_area

In [9]:
box1 = (2, 1, 4, 3)
box2 = (1, 2, 3, 4)

print("iou = " + str(IOU(box1, box2)))

iou = 0.14285714285714285


## YOLO Non Max Suppression

In [10]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes=10, iou_threshold=0.5):
    
    max_box_tensor = tf.Variable(max_boxes, dtype='int32')
    #tf.variables_initializer([max_boxes_tensor])
    
    nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes, iou_threshold)
    
    scores = tf.keras.backend.gather(scores, nms_indices)
    boxes = tf.keras.backend.gather(boxes, nms_indices)
    classes = tf.keras.backend.gather(classes, nms_indices)
    return scores, boxes, classes

In [11]:
scores = tf.random.normal([54, ], mean=1, stddev=4, seed=1)
boxes = tf.random.normal([54, 4], mean=1, stddev=4, seed=1)
classes = tf.random.normal([54,], mean=1, stddev=4, seed=1)
scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes)

In [12]:
print("scores[2] = " + str(tf.keras.backend.eval(scores[2])))
print("boxes[2] = " + str(tf.keras.backend.eval(boxes[2])))
print("classes[2] = " + str(tf.keras.backend.eval(classes[2])))
print("scores.shape = " + str(scores.shape))
print("boxes.shape = " + str(boxes.shape))
print("classes.shape = " + str(classes.shape))

scores[2] = 6.082263
boxes[2] = [ 4.110873   2.544206   2.1469054 -1.5198021]
classes[2] = 5.049652
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)


In [13]:
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

In [14]:
def yolo_eval(
    yolo_outputs, image_shape=(720., 1280.), max_boxes=10, score_threshold=0.6, iou_threshold=0.5
):
    box_confidence, box_xy, box_wh, box_class_probs = (yolo_outputs[0], yolo_outputs[1], yolo_outputs[2], yolo_outputs[3])
    boxes = yolo_boxes_to_corners(box_xy, box_wh)
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, score_threshold)
    boxes = scale_boxes(boxes, image_shape)
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes, iou_threshold)
    
    return scores, boxes, classes

In [15]:
yolo_outputs = (tf.random.normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1),
                    tf.random.normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                    tf.random.normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                    tf.random.normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1))
scores, boxes, classes = yolo_eval(yolo_outputs)
    

(19, 19, 5, 80) (19, 19, 5) (19, 19, 5) (19, 19, 5) (19, 19, 5, 4)


In [16]:
print("scores[2] = " + str(tf.keras.backend.eval(scores[2])))
print("boxes[2] = " + str(tf.keras.backend.eval(boxes[2])))
print("classes[2] = " + str(tf.keras.backend.eval(classes[2])))
print("scores.shape = " + str(scores.shape))
print("boxes.shape = " + str(boxes.shape))
print("classes.shape = " + str(classes.shape))

scores[2] = 147.72488
boxes[2] = [2651.31   1417.04   1748.9335 7298.7256]
classes[2] = 74
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)


In [17]:
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
image_shape = (720., 1280.,)

In [18]:
class_names
len(class_names)

80

In [19]:
from yolov3_tf2.models import YoloV3

In [20]:
yolo = YoloV3(classes=len(class_names))
yolo.load_weights("../tools/yolov3-tf2/yolov3.tf").expect_partial()

In [21]:
class_names = [c.strip() for c in class_names]

In [22]:
!pwd

/Users/shankar/dev/code/ds/studies/data_science/applications/car-detection


In [23]:
img_raw = tf.image.decode_image(open("../tools/yolov3-tf2/data/street.jpg", 'rb').read(), channels=3)

In [24]:
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset


In [25]:
img = tf.expand_dims(img_raw, 0)
img = transform_images(img, 416)

In [26]:
boxes, scores, classes, nums = yolo(img)

In [27]:
#yolo_model = load_model("model_data/yolo.h5")
from yolov3_tf2.utils import draw_outputs

In [29]:
import cv2
img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
img = draw_outputs(img, (boxes, scores, classes, nums), class_names)
cv2.imwrite("out.jpg", img)

True

In [30]:
yolo.summary()

Model: "yolov3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, None, None,  0                                            
__________________________________________________________________________________________________
yolo_darknet (Model)            ((None, None, None,  40620640    input[0][0]                      
__________________________________________________________________________________________________
yolo_conv_0 (Model)             (None, None, None, 5 11024384    yolo_darknet[1][2]               
__________________________________________________________________________________________________
yolo_conv_1 (Model)             (None, None, None, 2 2957312     yolo_conv_0[1][0]                
                                                                 yolo_darknet[1][1]          